In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import numpy as np
import pandas as pd

#images
import cv2

#modeling
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.client import device_lib

#visualizations
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
#Check to see
tf.config.get_visible_devices()

In [ ]:
train = pd.read_csv("/kaggle/input/petfinder-pawpularity-score/train.csv")
test = pd.read_csv("/kaggle/input/petfinder-pawpularity-score/train.csv")

In [ ]:
#Modify the Id such that each Id is the full image path. In the form
def train_id_to_path(x):
    return '/kaggle/input/petfinder-pawpularity-score/train/' + x + ".jpg"
def test_id_to_path(x):
    return '/kaggle/input/petfinder-pawpularity-score/test/' + x + ".jpg"


#Read in the data and drop unnecessary columns
train = pd.read_csv('/kaggle/input/petfinder-pawpularity-score/train.csv')
train = train.drop(['Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory', 'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur'],axis=1)

test = pd.read_csv('/kaggle/input/petfinder-pawpularity-score/test.csv')
test = test.drop(['Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory', 'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur'],axis=1)

#Add the .jpg extensions to the image file name ids
train["img_path"] = train["Id"].apply(train_id_to_path)
test["img_path"] = test["Id"].apply(test_id_to_path)

In [ ]:
train['two_bin_pawp'] = pd.qcut(train['Pawpularity'], q=2, labels=False)
train = train.astype({"two_bin_pawp": str})

train['four_bin_pawp'] = pd.qcut(train['Pawpularity'], q=4, labels=False)
train = train.astype({"four_bin_pawp": str})

train['ten_bin_pawp'] = pd.qcut(train['Pawpularity'], q=10, labels=False)
train = train.astype({"ten_bin_pawp": str})

In [ ]:
train2bin_stats = train.groupby('two_bin_pawp')
train2bin_stats.describe()

In [ ]:
train.head()

In [ ]:
#Set the size image you want to use
image_height = 128
image_width = 128

#define a function that accepts an image url and outputs an eager tensor
def path_to_eagertensor(image_path):
    raw = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(raw, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    #image = tf.image.resize_with_pad(image, image_height, image_width) #optional with padding to retain original dimensions
    image = tf.image.resize(image, (image_height, image_width))
    return image

In [ ]:
og_example_image = plt.imread('../input/petfinder-pawpularity-score/train/0007de18844b0dbbb5e1f607da0606e0.jpg') 
print(og_example_image.shape)

#then plt.imshow() can display it for you
plt.imshow(og_example_image)
plt.title('First Training Image') 
plt.axis('off') #turns off the gridlines
plt.show()

In [ ]:
example_image = path_to_eagertensor('../input/petfinder-pawpularity-score/train/0007de18844b0dbbb5e1f607da0606e0.jpg')
#show the type 
print('type: ', type(example_image),'\n shape: ',example_image.shape)
plt.imshow(example_image)
plt.title('First Training Image - with preprocessing done by path_to_eagertensor()') 
plt.show()

In [ ]:
#get all the images in the training folder and put their tensors in a list
X = []
for img in train['img_path']:
    new_img_tensor = path_to_eagertensor(img)
    X.append(new_img_tensor)
    
print(type(X),len(X))
X = np.array(X)
print(type(X),X.shape)

In [ ]:
# print(X[:1])

In [ ]:
#get all the images in the test folder and put their tensors in a list
X_submission = []
for img in test['img_path']:
    new_img_tensor = path_to_eagertensor(img)
    X_submission.append(new_img_tensor)
    
print(type(X_submission),len(X_submission))
X_submission = np.array(X_submission)
print(type(X_submission),X_submission.shape)

In [ ]:
# y = train['two_bin_pawp']
y = train["four_bin_pawp"]
print(y[:5])

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
print(y_test[:5])

In [ ]:
print(y_train[:5])

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Conv1D, MaxPooling1D
from keras.layers import Dense, Activation, Dropout, Flatten
from sklearn.datasets import make_blobs
from sklearn.preprocessing import MinMaxScaler

In [ ]:
model = Sequential()
# num_classes = 2

# # TODO: Conv1
model.add(Conv2D(filters=32, kernel_size=7,padding="same",activation="relu",input_shape=(128, 128, 3)))
model.add(MaxPooling2D(pool_size=7, strides=2, padding="valid"))
# # # TODO: Conv2
model.add(Conv2D(filters=16, kernel_size=5,padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=7, strides=2, padding="valid"))

# # # TODO: Conv3
model.add(Conv2D(filters=5, kernel_size=3,padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=7, strides=2, padding="valid"))
# # TODO: Flatten the layer
model.add(Flatten())

# # TODO: Add the intermediate fully connected layers (Dense in keras)
model.add(Dense(64, activation='relu'))

model.add(Dropout(0.5))
# # TODO: Add the final fully connected layer with the softmax activation function
model.add(Dense(4, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
#for two categories
# def convert2onehot(labels):
#   result = []
#   for label in labels:
# #         print(label)
#         if label == '1':
# #             print("label 1")
#             result.append([1, 0])
#         else:
# #             print("label 0")
#             result.append([0, 1])
#   return np.array(result)

# x_train_array = np.array(x_train)
# x_test_array = np.array(x_test)
# y_train_onehot = convert2onehot(y_train)
# y_test_onehot = convert2onehot(y_test)
# print(y_train[:5])
# print(y_train_onehot[:5])
# print(x_train_array[:5])

#for 4 categories 
def convert2onehot_4(labels):
  result = []
  for label in labels:
#         print(label)
        if label == '0':
#             print("label 1")
            result.append([1, 0, 0, 0])
        elif label == '1':
            result.append([0,1,0,0])
        elif label == '2':
            result.append([0,0,1,0])
        else:
#             print("label 0")
            result.append([0,0,0,1])
  return np.array(result)
x_train_array = np.array(x_train)
x_test_array = np.array(x_test)
y_train_onehot = convert2onehot_4(y_train)
y_test_onehot = convert2onehot_4(y_test)




In [ ]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy']) 
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) 

In [ ]:
print(x_train_array.shape, y_train_onehot.shape,x_test_array.shape, y_test_onehot.shape)

In [ ]:
model.fit(x_train_array, y_train_onehot, batch_size=16, epochs=3, validation_data=(x_test_array, y_test_onehot))